<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Data" data-toc-modified-id="Importing-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Data</a></span></li><li><span><a href="#Checking-for-duplicates" data-toc-modified-id="Checking-for-duplicates-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Checking for duplicates</a></span></li><li><span><a href="#Checking-for-outliers" data-toc-modified-id="Checking-for-outliers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Checking for outliers</a></span></li><li><span><a href="#Checking-for-silent-/-pause-tracks" data-toc-modified-id="Checking-for-silent-/-pause-tracks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Checking for silent / pause tracks</a></span></li><li><span><a href="#Checking-Nulls-in-'artist_genre'" data-toc-modified-id="Checking-Nulls-in-'artist_genre'-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Checking Nulls in 'artist_genre'</a></span></li><li><span><a href="#Enhancing-genre-contextual-information" data-toc-modified-id="Enhancing-genre-contextual-information-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Enhancing genre contextual information</a></span></li></ul></div>

In [6]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import itertools
from google.cloud import bigquery
from google.oauth2 import service_account

# NOTE: remove silent tracks!
# tracks under 10 secs?

PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


### Importing Data

In [7]:
credentials = service_account.Credentials.from_service_account_file(filename="spotify-324112-6c25e5f19c02.json")
bq = bigquery.Client(
           credentials=credentials,
           project=credentials.project_id,
        )

sql = """
    SELECT *
    FROM `spotify-324112.tracks.tblTracks`
    WHERE danceability IS NOT NULL;
"""

raw_df = bq.query(sql).to_dataframe()
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91662 entries, 0 to 91661
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         91662 non-null  object 
 1   artist_name       91662 non-null  object 
 2   id                91662 non-null  object 
 3   name              91662 non-null  object 
 4   release_date      91662 non-null  object 
 5   year              91662 non-null  int64  
 6   duration_ms       91662 non-null  int64  
 7   explicit          91662 non-null  int64  
 8   popularity        91662 non-null  int64  
 9   artist_genre      80041 non-null  object 
 10  danceability      91662 non-null  float64
 11  energy            91662 non-null  float64
 12  key               91662 non-null  int64  
 13  loudness          91662 non-null  float64
 14  mode              91662 non-null  int64  
 15  speechiness       91662 non-null  float64
 16  acousticness      91662 non-null  float6

In [4]:
raw_df.head()

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0HTZUrKoyhew3gmvCnnV37,李昕融,7qSsa5kvZndM8beDFJVxlE,你笑起來真好看,2019-04-24,2019,172032,0,42,None,...,0.448,3,-7.952,1,0.0444,0.6110,0.000,0.111,0.553,83.976
1,0RLNRNxmEHZN7EStjBf0O4,Lord Kael,32TW5KGM1UqTzkTOkcNITt,Sweetly,2019-07-01,2019,135515,0,65,None,...,0.401,3,-13.280,0,0.2870,0.6120,0.763,0.244,0.592,125.872
2,0rK0ZPLX4fKnFSbqs6gYfY,Blue.D,3U5ti2dwp5FA70lZPrhv9l,NOBODY,2019-12-02,2019,201183,0,53,None,...,0.426,5,-7.134,0,0.0437,0.5660,0.000,0.299,0.491,88.626
3,1aSNrbYVjARHVEAjIvaONY,李俊佑,34VJQsy24tqXzwtAqcPRhf,寵壞,2019-03-18,2019,196707,0,41,None,...,0.751,2,-5.246,1,0.0411,0.3370,0.000,0.242,0.559,143.993
4,1sdnOXw9epamqw5ERPK8T5,Valntn,2Rtj1B24RTIrIg2TMQ8WS9,Mona Lisa,2019-07-10,2019,157019,0,57,None,...,0.590,1,-5.140,0,0.0515,0.0493,0.000,0.280,0.787,119.934


In [5]:
raw_df.columns

Index(['artist_id', 'artist_name', 'id', 'name', 'release_date', 'year',
       'duration_ms', 'explicit', 'popularity', 'artist_genre', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

### Checking for duplicates

In [6]:
raw_df[raw_df.duplicated(keep=False)]

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
555,5Xd9nARqQJcNWKvzQb3sRp,Bhumiband,33nUtsIggPcS1jctWyTzPG,Goda,2006-04-01,2006,275388,0,21,None,...,0.772,4,-5.823,0,0.0528,0.0877,0.000001,0.0971,0.193,124.980
556,5Xd9nARqQJcNWKvzQb3sRp,Bhumiband,33nUtsIggPcS1jctWyTzPG,Goda,2006-04-01,2006,275388,0,21,None,...,0.772,4,-5.823,0,0.0528,0.0877,0.000001,0.0971,0.193,124.980
689,0nyc9SZGLITSOJASmTZsnZ,LEN,4agp6oHofabdUedr0B1krj,Steal My Sunshine,1999-05-25,1999,265533,0,60,None,...,0.829,4,-11.112,1,0.5290,0.3940,0.000000,0.1080,0.677,191.476
690,0nyc9SZGLITSOJASmTZsnZ,LEN,4agp6oHofabdUedr0B1krj,Steal My Sunshine,1999-05-25,1999,265533,0,60,None,...,0.829,4,-11.112,1,0.5290,0.3940,0.000000,0.1080,0.677,191.476
691,0nyc9SZGLITSOJASmTZsnZ,LEN,5Muvh0ooAJkSgBylFyI3su,Steal My Sunshine - Single Version,1999-09-06,1999,211466,0,56,None,...,0.944,4,-8.473,1,0.2490,0.5270,0.000000,0.2140,0.710,192.147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91620,0Mz5XE0kb1GBnbLQm2VbcO,Mos Def,5Ru9OjtEw5htfnVKpYsIcv,UMI Says,1999-10-12,1999,305280,1,56,"alternative hip hop,conscious hip hop,east coa...",...,0.670,2,-10.224,1,0.1320,0.3050,0.000000,0.0928,0.566,121.342
91636,1W9qOBYRTfP7HcizWN43G1,Slick Rick,4TGHOWSNAwUXn5onDQr0P1,Street Talkin',1999-01-01,1999,221200,1,59,"alternative hip hop,bronx hip hop,east coast h...",...,0.495,5,-7.483,0,0.3460,0.0544,0.000000,0.0934,0.836,101.781
91637,1W9qOBYRTfP7HcizWN43G1,Slick Rick,4TGHOWSNAwUXn5onDQr0P1,Street Talkin',1999-01-01,1999,221200,1,59,"alternative hip hop,bronx hip hop,east coast h...",...,0.495,5,-7.483,0,0.3460,0.0544,0.000000,0.0934,0.836,101.781
91649,78xUyw6FkVZrRAtziFdtdu,The Roots,4pdLZsxq0y5oJDb6Cxlokw,You Got Me,1999-02-23,1999,259306,1,65,"alternative hip hop,conscious hip hop,east coa...",...,0.442,11,-8.106,0,0.2850,0.1510,0.000000,0.1890,0.757,81.029


In [7]:
raw_df[raw_df.duplicated(subset=['artist_name','name'], keep=False)].sort_values('name')


,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
66805,2D7RkvtKKb6E5UmbjQM1Jd,Georges Bizet,0xVbtHC4BM071oCAYrMxy1,"""Carmen Fantasie"", by Franz Waxman, based on t...",1946,1946,583573,0,0,"classical,french opera,french romanticism,late...",...,0.198,9,-19.978,1,0.0539,0.94300,0.376000,0.1940,0.342,70.255
66813,2D7RkvtKKb6E5UmbjQM1Jd,Georges Bizet,5AZvNKWS9yPpXntKqHAhYw,"""Carmen Fantasie"", by Franz Waxman, based on t...",1934,1934,570920,0,1,"classical,french opera,french romanticism,late...",...,0.193,2,-20.298,1,0.0457,0.94100,0.433000,0.4150,0.330,131.083
31873,4UXqAaa6dQYAk18Lv7PEgX,Fall Out Boy,4zCfMDdf5QXPKEqxdinXvB,"""The Take Over, The Breaks Over""",2007-02-06,2007,213586,0,50,"emo,modern rock,pop punk",...,0.917,9,-2.563,1,0.0477,0.00614,0.000020,0.0775,0.670,149.948
31868,4UXqAaa6dQYAk18Lv7PEgX,Fall Out Boy,3rG8ZkmKHb4Ms6CsSzEITv,"""The Take Over, The Breaks Over""",2007-01-01,2007,213586,0,57,"emo,modern rock,pop punk",...,0.917,9,-2.563,1,0.0477,0.00614,0.000020,0.0775,0.670,149.948
34522,1dfeR4HaWDbWqFHLkxsg1d,Queen,6aNP9GlBi3VHPXl7w3Qjr9,'39 - Remastered 2011,1975-11-21,1975,210800,0,43,"classic rock,glam rock,rock",...,0.571,8,-9.686,1,0.0273,0.02710,0.000000,0.1110,0.373,101.626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52097,2eVlgLy3Aym09gM3dqx6cq,Lee Moon Sae,4bIXDxPN1ddswIpU8WstIT,옛사랑,1999-05-01,1999,277560,0,35,"classic korean pop,korean pop,world worship",...,0.301,9,-13.614,1,0.0456,0.80400,0.000343,0.9530,0.191,126.705
22048,5SnaL8SsjGMHQNyqpa8Zos,2AM,6bX31xvB4HeeUOMorR77oV,죽어도 못 보내,2010-01-21,2010,195905,0,40,"k-pop,korean pop",...,0.776,2,-3.102,1,0.0323,0.21000,0.000000,0.1450,0.540,136.831
22047,5SnaL8SsjGMHQNyqpa8Zos,2AM,6PHctRrk4oCNORizN1TEbe,죽어도 못 보내,2010-03-16,2010,195906,0,44,"k-pop,korean pop",...,0.772,2,-3.093,1,0.0300,0.20700,0.000000,0.1510,0.543,136.857
848,3QUst1HlAmRuM48c5n5HSB,Park Wan Kyu,36HpeuZMRWKAyVaRO8mePe,천년의 사랑,1999-09-08,1999,265534,0,30,None,...,0.559,9,-6.389,0,0.0317,0.07850,0.000000,0.0517,0.153,133.916


* After investigating, the reason for the duplication is due to the same song being released in different albums. 
* With respect to recommendation systems, duplicate songs are not ideal since we do not want to recommend the same song again
* Hence, we are dropping duplicates.

In [8]:
raw_df = raw_df.drop_duplicates(subset=['artist_name','name'])
print(raw_df.shape)

(85335, 21)


### Checking for outliers

In [9]:
raw_df.describe()

,year,duration_ms,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,85335.000000,8.533500e+04,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000,85335.000000
mean,1975.069444,2.318282e+05,0.028054,30.367399,0.540059,0.441834,5.153138,-11.878144,0.696033,0.136625,0.557620,0.142408,0.196453,0.510384,115.688813
std,26.245854,1.302971e+05,0.165129,22.480527,0.167140,0.252129,3.496931,5.747152,0.459971,0.251060,0.352938,0.296702,0.156444,0.257533,30.313925
min,1930.000000,6.466000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1952.000000,1.791225e+05,0.000000,6.000000,0.425000,0.238000,2.000000,-15.264000,0.000000,0.033500,0.211000,0.000000,0.101000,0.301000,92.205000
50%,1975.000000,2.086000e+05,0.000000,32.000000,0.556000,0.408000,5.000000,-11.025000,1.000000,0.042900,0.627000,0.000040,0.135000,0.508000,114.044000
75%,1998.000000,2.612815e+05,0.000000,48.000000,0.667000,0.635000,8.000000,-7.543000,1.000000,0.073200,0.900000,0.030450,0.248000,0.719000,133.992500
max,2020.000000,7.054236e+06,1.000000,98.000000,0.988000,1.000000,11.000000,4.584000,1.000000,0.970000,0.996000,1.000000,0.996000,1.000000,243.372000


Observations:
*  Zero tempo is quite unexpected. Worth investigating.


In [10]:
raw_df[raw_df['tempo'] == 0].head(100)

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
25,71JAcPfeNMu1clNa9ifPUx,Cover Kid,3mlNNPtgbnhrrBUxSOIJKK,You Are My Sunshine,2019-08-09,2019,85500,0,53,None,...,0.03270,9,-28.065,1,0.0,0.995,0.94700,0.0985,0.0,0.0
271,4POPMvpDC7yGtZZ1agfhT0,Torsten Abrolat,4sCHTxLELUw6Gpgk9lkMz5,"Fön Geräusch (Konstantes, warmes wohliges Raus...",2012-11-27,2012,358264,0,50,None,...,0.00011,0,-6.022,1,0.0,0.123,0.99600,0.9760,0.0,0.0
6197,4C7UikoKpiwbAiGV89FRVh,Master Ebrahim,21zsEwpxCGDFbEzqzuDP5t,O Duniya Ke Rakhwale - Instrumental,1944-12-01,1944,172130,0,0,None,...,0.30100,7,-11.738,0,0.0,0.983,0.47700,0.0917,0.0,0.0
6435,2h9qg2PnZvd0ZOOXHw2ex7,Samuel Gardner,2XshGbcXScAhxsLwUy3SVc,"From the Canebrake, Op. 5, No. 1",1946-01-01,1946,90146,0,0,None,...,0.18300,2,-10.689,1,0.0,0.971,0.01110,0.1400,0.0,0.0
7205,20X1HNJbAFOW2hIv9LUipj,The Slobs,0WbcA0RsyG8KntmSCnByR8,The Christmas Raid,1941,1941,10370,0,0,None,...,0.37300,1,-12.173,1,0.0,0.583,0.97900,0.3590,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81282,1pBuKaLHJlIlqYxQQaflve,Benny Goodman,3IcXTeq9O2dpsSXsDj9naH,Pause Track - Live,1938,1938,6466,0,0,"adult standards,big band,jazz,jazz clarinet,lo...",...,0.00000,0,-60.000,0,0.0,0.000,0.00000,0.0000,0.0,0.0
82540,6KWcxMWVNVIYbdOQyJtsSy,The Platters,2pvOAI0XtZeAQFAko9TdNE,A Tisket A Tasket,1959-01-01,1959,127093,0,16,"adult standards,brill building pop,doo-wop,lou...",...,0.49500,8,-11.744,1,0.0,0.747,0.00000,0.1590,0.0,0.0
83256,3EY5DxGdy7x4GelivOjS2Q,Connie Francis,0zr7DJGTPUfAUmjM7crmt2,Hava Nagilah,1960-01-01,1960,179466,0,21,"adult standards,brill building pop,easy listen...",...,0.00000,0,-60.000,0,0.0,0.000,0.00000,0.0000,0.0,0.0
84052,19eLuQmk9aCobbVDHc6eek,Louis Armstrong,63kd4m3VFxcJjPVVtbVNAu,"Hello, Dolly!",1964-10-25,1964,147000,0,51,"adult standards,dixieland,harlem renaissance,j...",...,0.40500,0,-9.935,1,0.0,0.842,0.00114,0.1980,0.0,0.0


* The audio features play a really critical part in decision process.
* Given it makes a very small part of the data, it should be ok to drop these rows

In [11]:
raw_df = raw_df[raw_df['tempo'] != 0]
print(raw_df.shape)

(85263, 21)


In [12]:
raw_df.describe()

,year,duration_ms,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,85263.000000,8.526300e+04,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000,85263.000000
mean,1975.060612,2.318684e+05,0.028078,30.364765,0.540515,0.442035,5.153150,-11.870447,0.696070,0.136740,0.557551,0.142090,0.196393,0.510815,115.786505
std,26.236851,1.297679e+05,0.165196,22.476684,0.166472,0.252016,3.496712,5.731422,0.459956,0.251134,0.352909,0.296383,0.156351,0.257214,30.139648
min,1930.000000,1.530600e+04,0.000000,0.000000,0.057400,0.000020,0.000000,-47.600000,0.000000,0.022000,0.000001,0.000000,0.011400,0.000000,30.946000
25%,1952.000000,1.791955e+05,0.000000,6.000000,0.425000,0.238000,2.000000,-15.256000,0.000000,0.033600,0.211000,0.000000,0.101000,0.302000,92.264000
50%,1975.000000,2.086530e+05,0.000000,32.000000,0.556000,0.409000,5.000000,-11.021000,1.000000,0.042900,0.627000,0.000040,0.135000,0.508000,114.072000
75%,1998.000000,2.613360e+05,0.000000,48.000000,0.667000,0.635000,8.000000,-7.540000,1.000000,0.073300,0.900000,0.030100,0.248000,0.720000,134.001000
max,2020.000000,7.054236e+06,1.000000,98.000000,0.988000,1.000000,11.000000,4.584000,1.000000,0.970000,0.996000,0.999000,0.996000,1.000000,243.372000


In [13]:
raw_df.sort_values('duration_ms').head()

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
81278,1pBuKaLHJlIlqYxQQaflve,Benny Goodman,2H7m6mWjXbNBH6vspeBt9K,Conclusion - Live,1938,1938,15306,0,0,"adult standards,big band,jazz,jazz clarinet,lo...",...,0.274,0,-22.603,0,0.952,0.96300,0.000000,0.131,0.737,169.654
81297,1pBuKaLHJlIlqYxQQaflve,Benny Goodman,0KNql3hCgwu1dpScGv4zag,Twenty Years of Jazz - Live,1938,1938,15426,0,0,"adult standards,big band,jazz,jazz clarinet,lo...",...,0.114,1,-21.184,1,0.958,0.96400,0.000000,0.140,0.884,181.422
23862,6AmG7SNdlV9boPZ7dFVnLY,Charlie Chaplin,0qCLQ6gNLruIT9gwA39H6I,Spring Song with Banjo,1952,1952,16416,0,0,classic soundtrack,...,0.631,10,-16.191,1,0.105,0.99500,0.881000,0.271,0.986,95.520
81302,1pBuKaLHJlIlqYxQQaflve,Benny Goodman,26AUxgMYVZkahwZsxnpuWs,Swingtime In the Rockies - Live,1938,1938,16640,0,0,"adult standards,big band,jazz,jazz clarinet,lo...",...,0.384,1,-21.278,1,0.953,0.98200,0.000332,0.129,0.635,156.661
2332,1DAxGq6wVo10yU3vjZV9kM,Unspecified,02QuCbbUN4o5qevd1ofIiW,Man Made Sounds - Jet Flight,1953-01-01,1953,17186,0,0,None,...,0.779,7,-14.765,1,0.047,0.00492,0.756000,0.488,0.000,85.730


### Checking for silent / pause tracks

In [14]:
print(raw_df[raw_df['name'].str.lower().str.contains('pause')].shape)
print(raw_df[raw_df['name'].str.lower().str.contains('silent')].shape)

(1, 21)
(28, 21)


In [15]:
display(raw_df[raw_df['name'].str.lower().str.contains('pause')].head())
display(raw_df[raw_df['name'].str.lower().str.contains('silent')].head(20))

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
88360,6Mo9PoU6svvhgEum7wh2Nd,Public Enemy,1zONdCQrEJ4t3mlzKQADEN,Rebel Without A Pause,1988-06-28,1988,302800,0,49,"conscious hip hop,east coast hip hop,gangster ...",...,0.692,7,-13.906,1,0.346,0.0127,0.0,0.69,0.651,99.771


,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
573,6ZyuwTFPiSQVgnjVB2264q,Eric Chiryoku,5B2EG1dwOkiXICgnnmptTr,Silent Wind,2007-01-31,2007,302666,0,19,None,...,0.2600,11,-9.484,0,0.0350,0.920,0.966000,0.1140,0.1740,83.546
940,29M6fB5DXQGVJYsx66rDPo,Cagnet,7iS4rTNs8f9TpJtEdeJVCl,Silent Emotion,1996-05-29,1996,245333,0,23,None,...,0.3880,8,-15.273,0,0.0290,0.335,0.726000,0.0974,0.4130,105.138
2001,395Z91yDQ05pkMbRKik18y,Franz Xaver Gruber,0q09aiEyRnPxnv2svUn6CY,Gruber: Silent Night,1952-01-01,1952,129354,0,0,None,...,0.2270,7,-15.633,1,0.0536,0.642,0.000000,0.1100,0.3540,71.040
11810,117YLv0bvUKDzQgaHwTF7b,Spike Jones,4jDI1fxW7OF8W3I4xUJz44,Silent Night,1956-01-01,1956,163440,0,1,comic,...,0.2520,0,-10.925,1,0.0342,0.986,0.044200,0.1230,0.0627,111.390
19116,1ZoSXXLPsWBwK2VmIvny8A,Ken Elkinson,2KEZNLyReIWxzkwqTWVbVm,Silent Night,1941,1941,246173,0,0,new age piano,...,0.0330,0,-24.215,1,0.0463,0.995,0.945000,0.1100,0.2670,67.838
20830,5f4UBNURyjjZJDqhLCCfmC,Tony O'Connor,6sXAaxtCZFjcUPKX7z1UtS,Silent Spirit,1991-01-01,1991,457960,0,42,"healing,new age",...,0.0639,6,-23.746,1,0.0453,0.923,0.121000,0.0878,0.1470,81.804
32560,43ZHCT0cAZBISjO8DG9PnE,Elvis Presley,6cw1OgKsuEWQbmQb5Z4a3T,Silent Night,1957-10-15,1957,145000,0,27,"rock-and-roll,rockabilly",...,0.0314,10,-23.411,1,0.0351,0.960,0.067800,0.1130,0.1760,135.490
35882,4aP1lp10BRYZO658B2NwkG,Andrew Lloyd Webber,1z2oyr9Uu9GCz31kycvPuX,Masquerade / Why So Silent,1987-01-01,1987,383533,0,35,"broadway,show tunes,west end",...,0.2830,0,-14.929,1,0.0587,0.766,0.000000,0.2610,0.3320,130.502
41087,5RdqZVi36tpDPYNPw8jJbO,Julie Andrews,5hgkmxlHJdClFuOEF26kzJ,"Silent Night, Holy Night",1949,1949,141960,0,17,"hollywood,movie tunes,show tunes",...,0.1330,10,-19.422,1,0.0393,0.851,0.000206,0.1780,0.1190,88.239
44747,4iHNK0tOyZPYnBU7nGAgpQ,Mariah Carey,1sFWEpf1aPYN576LS1aa4Y,Silent Night,1994-11-01,1994,219293,0,39,"dance pop,pop,r&b,urban contemporary",...,0.3630,10,-11.667,1,0.0343,0.812,0.000000,0.1130,0.2360,174.903


* Seems like there are no silent / pause tracks which is good

### Checking Nulls in 'artist_genre'

* artist_genre is probably going to be quite important. 
* Will try and see if we can use the audio features to cover up for the lack of genre

In [16]:
raw_df[raw_df['artist_genre'].isnull()]

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0HTZUrKoyhew3gmvCnnV37,李昕融,7qSsa5kvZndM8beDFJVxlE,你笑起來真好看,2019-04-24,2019,172032,0,42,None,...,0.448,3,-7.952,1,0.0444,0.6110,0.000000,0.1110,0.553,83.976
1,0RLNRNxmEHZN7EStjBf0O4,Lord Kael,32TW5KGM1UqTzkTOkcNITt,Sweetly,2019-07-01,2019,135515,0,65,None,...,0.401,3,-13.280,0,0.2870,0.6120,0.763000,0.2440,0.592,125.872
2,0rK0ZPLX4fKnFSbqs6gYfY,Blue.D,3U5ti2dwp5FA70lZPrhv9l,NOBODY,2019-12-02,2019,201183,0,53,None,...,0.426,5,-7.134,0,0.0437,0.5660,0.000000,0.2990,0.491,88.626
3,1aSNrbYVjARHVEAjIvaONY,李俊佑,34VJQsy24tqXzwtAqcPRhf,寵壞,2019-03-18,2019,196707,0,41,None,...,0.751,2,-5.246,1,0.0411,0.3370,0.000000,0.2420,0.559,143.993
4,1sdnOXw9epamqw5ERPK8T5,Valntn,2Rtj1B24RTIrIg2TMQ8WS9,Mona Lisa,2019-07-10,2019,157019,0,57,None,...,0.590,1,-5.140,0,0.0515,0.0493,0.000000,0.2800,0.787,119.934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89152,5Gvv5mwn9h9YFegYfXMfGk,Kim & Susan,1ePMtZfGQ4Kdixc8SsUCQA,As time goes by,1931,1931,190641,1,4,None,...,0.400,2,-11.648,0,0.0319,0.8200,0.000296,0.1770,0.313,68.854
89153,00VkipgdR6QGtdKPpXxuWA,Junkie Ravher,2WLeWRNH9kBXoWYfkHfz74,Concentrado en Lo Mio,1930-03-10,1930,230952,1,0,None,...,0.498,0,-9.299,0,0.7220,0.1010,0.000000,0.0936,0.677,169.888
89154,6En6aS2zQyNyTI3pWfjlNR,Shorty Bralik,0yZF0h8t110TJduzY3dLI2,Gang Bang,1930-05-29,1930,175281,1,0,None,...,0.599,2,-8.286,1,0.4160,0.3230,0.000000,0.0883,0.890,87.735
89155,6En6aS2zQyNyTI3pWfjlNR,Shorty Bralik,1BYj2lRr4DiQbuJKQ13MO9,Up There,1930-05-29,1930,229929,1,0,None,...,0.518,5,-9.919,0,0.4130,0.6010,0.000000,0.0840,0.369,77.285


### Enhancing genre contextual information

In [42]:
test_df = raw_df[raw_df['artist_genre'].notnull()]
lst = test_df['year'].unique().tolist()
lst1 = sorted(list(set([str(item)[:3] + '0s' for item in lst])))
print(lst)
print(lst1)

[1987, 1980, 1978, 1982, 1986, 1989, 1996, 1983, 1991, 1971, 2019, 2018, 2017, 2016, 2011, 2013, 2015, 2012, 2014, 2002, 2004, 2005, 2003, 2009, 2006, 2010, 2008, 2007, 2000, 2020, 1985, 1972, 1976, 1998, 1979, 1973, 1975, 1974, 1981, 1968, 1953, 1997, 1954, 1955, 1951, 1957, 1958, 1944, 1995, 1966, 1952, 1956, 1950, 1959, 1941, 1930, 2001, 1999, 1988, 1977, 1984, 1990, 1970, 1992, 1993, 1994, 1947, 1948, 1949, 1960, 1969, 1942, 1943, 1946, 1935, 1965, 1967, 1961, 1963, 1964, 1940, 1939, 1945, 1934, 1962, 1932, 1938, 1933, 1936, 1937, 1931]
['1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']


In [65]:
lst = ['a', 'b', 'c']
lst1 = [''] + lst
print(lst1)
del lst1[0]

print(lst1)

['', 'a', 'b', 'c']
['a', 'b', 'c']


In [1]:
# test_df = raw_df[raw_df['artist_genre'].notnull()]['artist_genre'].str.split(',')
# test_df = test_df.explode('artist_genre').drop_duplicates()
# print(np.sort(test_df.unique()).tolist())
# test_df = test_df.sort_values()
# #test_df.to_csv('genres.csv', index=False)
!pip install streamlit-player

In [17]:
genre_df = pd.read_csv('data/data_by_genres.csv')
genre_df.head()

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,context,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.166667,5,1,432hz,NaN,NaN,NaN,NaN,NaN
1,[],0.679245,0.521473,2.298526e+05,0.401522,0.196737,0.221586,-12.679076,0.112267,112.025168,0.515920,21.682005,7,1,[],NaN,NaN,NaN,NaN,NaN
2,a cappella,0.621532,0.577017,1.936522e+05,0.345694,0.003799,0.127087,-12.770211,0.095324,111.813230,0.453186,43.351819,11,1,acappella,NaN,NaN,NaN,NaN,NaN
3,abstract,0.359395,0.459500,3.430185e+05,0.487000,0.791400,0.119480,-14.092000,0.043420,124.743200,0.304990,41.500000,1,1,abstract,NaN,NaN,NaN,NaN,NaN
4,abstract beats,0.353347,0.694400,2.338244e+05,0.613400,0.349403,0.102453,-6.699800,0.143453,119.398400,0.634187,58.600000,10,0,abstract beats,NaN,NaN,NaN,NaN,NaN


In [18]:
merged_df = raw_df.copy()
merged_df['artist_genre'] = merged_df['artist_genre'].str.split(',')
merged_df = merged_df.explode('artist_genre')
merged_df.sort_values('artist_genre').head()

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
42725,06SaL19JfXGCw4nKilKfoL,PowerThoughts Meditation Club,6gvA6dned1EoET07mCvgPF,432Hz Miracle Tone: Shamanic Consciousness (Th...,2000,2000,1873000,0,36,432hz,...,0.0154,10,-31.861,1,0.0467,0.903,0.964000,0.0963,0.037,65.164
48011,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,3HAN4cAAtFwz7t8qYE38DU,Bellas Regionals: The Sign/Eternal Flame/Turn ...,2012-01-01,2012,160013,0,53,a cappella,...,0.2980,7,-6.487,1,0.0353,0.868,0.000000,0.2590,0.507,126.265
48010,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,3KTqBGgBfT7zzHSSj0gvP5,Pool Mashup: Just The Way You Are/Just A Dream,2012-01-01,2012,98560,0,54,a cappella,...,0.2950,6,-7.269,1,0.0336,0.909,0.000999,0.0591,0.670,109.706
55916,1T81IFRFywCJW0SNWMv3T1,The Treblemakers,7zgveKrOxukG95vxIO3KHd,Trebles Finals: Bright Lights Bigger City/Magic,2012-01-01,2012,154560,0,51,a cappella,...,0.5250,9,-7.257,0,0.2340,0.201,0.000000,0.0826,0.465,112.948
48009,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,5DnDWdt25emVC9CLUi3Iku,Bellas Finals: Price Tag/Don’t You (Forget Abo...,2012-01-01,2012,217080,0,58,a cappella,...,0.6910,0,-5.784,1,0.1770,0.569,0.000000,0.0904,0.275,105.335


In [19]:
merged_df = genre_df[['genres', 'context']].merge(merged_df, how='right', left_on='genres', right_on='artist_genre' ).drop(['genres'], axis='columns')
display(merged_df.sort_values('artist_genre').head(20))

,context,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
135845,432hz,06SaL19JfXGCw4nKilKfoL,PowerThoughts Meditation Club,6gvA6dned1EoET07mCvgPF,432Hz Miracle Tone: Shamanic Consciousness (Th...,2000,2000,1873000,0,36,...,0.0154,10,-31.861,1,0.0467,0.90300,0.964000,0.0963,0.037,65.164
142796,acappella,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,3HAN4cAAtFwz7t8qYE38DU,Bellas Regionals: The Sign/Eternal Flame/Turn ...,2012-01-01,2012,160013,0,53,...,0.2980,7,-6.487,1,0.0353,0.86800,0.000000,0.2590,0.507,126.265
142794,acappella,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,3KTqBGgBfT7zzHSSj0gvP5,Pool Mashup: Just The Way You Are/Just A Dream,2012-01-01,2012,98560,0,54,...,0.2950,6,-7.269,1,0.0336,0.90900,0.000999,0.0591,0.670,109.706
162041,acappella,1T81IFRFywCJW0SNWMv3T1,The Treblemakers,7zgveKrOxukG95vxIO3KHd,Trebles Finals: Bright Lights Bigger City/Magic,2012-01-01,2012,154560,0,51,...,0.5250,9,-7.257,0,0.2340,0.20100,0.000000,0.0826,0.465,112.948
142792,acappella,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,5DnDWdt25emVC9CLUi3Iku,Bellas Finals: Price Tag/Don’t You (Forget Abo...,2012-01-01,2012,217080,0,58,...,0.6910,0,-5.784,1,0.1770,0.56900,0.000000,0.0904,0.275,105.335
219786,acappella,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,44GrIJK7WaW0KXUUxg9qSl,Chanson d'Amour,1976-08-19,1976,173280,0,41,...,0.3030,2,-17.040,0,0.0261,0.02370,0.000308,0.1700,0.564,98.918
219783,acappella,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,3NMvVmKyjNGs91aSYje0RZ,Tuxedo Junction,1975-04-02,1975,185600,0,43,...,0.3580,1,-9.364,0,0.0412,0.68200,0.000005,0.1360,0.470,111.676
142788,acappella,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,1est72o2prNA80flC5ncoN,Party In The U.S.A.,2012-01-01,2012,63080,0,53,...,0.3600,10,-8.575,0,0.1050,0.82200,0.000000,0.1770,0.700,97.420
219795,acappella,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,4H6EV7soFKhurMK2CFfpmM,Boy From New York City,1981-06-01,1981,222160,0,39,...,0.6330,2,-9.376,1,0.0531,0.45700,0.000015,0.0967,0.702,151.522
142786,acappella,2BaKphge9AdjZIxQqv7WoQ,The Barden Bellas,6Gds0xbRf66IPJ3Qo0pmPL,Riff Off: Mickey/Like A Virgin/Hit Me With You...,2012-01-01,2012,223840,0,56,...,0.4990,3,-5.586,0,0.3550,0.57600,0.000000,0.1010,0.706,125.492


In [20]:
merged_df = pd.pivot_table(data=merged_df, index=['id'], values=['context'], aggfunc=list)
merged_df['context'] = merged_df['context'].map(lambda x: list(set(list(itertools.chain(x)))))
merged_df.head()

,context
id,
00094lLv0fy87cC0FFwDVh,[nan]
000GhijgPmcM1uikNOvclJ,[nan]
000Npgk5e2SgwGaIsN3ztv,"[filmi, classic pakistani pop, ghazal, sufi, c..."
000ZxLGm7jDlWCHtcXSeBe,"[boogie woogie blues, piano blues, stride]"
000jBcNljWTnyjB4YO7ojf,[nan]


In [21]:
new_df = raw_df.merge(merged_df, how='left', on='id')
print(new_df.info())
new_df.sort_values('artist_genre').head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85263 entries, 0 to 85262
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         85263 non-null  object 
 1   artist_name       85263 non-null  object 
 2   id                85263 non-null  object 
 3   name              85263 non-null  object 
 4   release_date      85263 non-null  object 
 5   year              85263 non-null  int64  
 6   duration_ms       85263 non-null  int64  
 7   explicit          85263 non-null  int64  
 8   popularity        85263 non-null  int64  
 9   artist_genre      74030 non-null  object 
 10  danceability      85263 non-null  float64
 11  energy            85263 non-null  float64
 12  key               85263 non-null  int64  
 13  loudness          85263 non-null  float64
 14  mode              85263 non-null  int64  
 15  speechiness       85263 non-null  float64
 16  acousticness      85263 non-null  float6

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,context
40822,06SaL19JfXGCw4nKilKfoL,PowerThoughts Meditation Club,6gvA6dned1EoET07mCvgPF,432Hz Miracle Tone: Shamanic Consciousness (Th...,2000,2000,1873000,0,36,432hz,...,10,-31.861,1,0.0467,0.90300,0.964000,0.0963,0.0370,65.164,[432hz]
42157,26AHtbjWKiwYzsoGoUZq53,Pentatonix,7dJGehjbhJvs3K4fWwYTW1,Radioactive,2014-09-19,2014,226440,0,58,a cappella,...,8,-7.642,1,0.0695,0.49000,0.000000,0.2980,0.4640,135.007,[acappella]
42158,2FjkZT851ez950cyPjeYid,Bobby McFerrin,5YbgcwHjQhdT1BYQ4rxWlD,Don't Worry Be Happy,1997-01-01,1997,292400,0,66,a cappella,...,11,-22.597,1,0.1650,0.88100,0.000076,0.0688,0.7710,69.283,[acappella]
42156,26AHtbjWKiwYzsoGoUZq53,Pentatonix,1klGbW5a9qTBFUjFfddbmU,Can't Sleep Love,2015-10-16,2015,173053,0,58,a cappella,...,2,-7.050,1,0.0455,0.08490,0.000000,0.1690,0.5720,94.596,[acappella]
42155,26AHtbjWKiwYzsoGoUZq53,Pentatonix,6ukvsBzq4d1vBsAUmz7ZVt,Daft Punk,2013-11-05,2013,248560,0,57,a cappella,...,11,-7.588,0,0.0605,0.00403,0.000000,0.1440,0.7200,133.014,[acappella]
42154,26AHtbjWKiwYzsoGoUZq53,Pentatonix,550rQQCGkrTzvp4SfpOPzx,Hallelujah,2016-10-21,2016,268960,0,63,a cappella,...,7,-7.385,0,0.0356,0.45400,0.000000,0.3380,0.3660,118.669,[acappella]
67000,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,2GOSOphMbrM7m2hfGjfgzz,Smile Again,1981-06-01,1981,276600,0,36,"a cappella,adult standards,vocal jazz",...,6,-12.578,0,0.0305,0.62100,0.000002,0.1710,0.1250,119.804,"[vocal jazz, adult standards, acappella]"
66998,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,44GrIJK7WaW0KXUUxg9qSl,Chanson d'Amour,1976-08-19,1976,173280,0,41,"a cappella,adult standards,vocal jazz",...,2,-17.040,0,0.0261,0.02370,0.000308,0.1700,0.5640,98.918,"[vocal jazz, adult standards, acappella]"
66997,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,3NMvVmKyjNGs91aSYje0RZ,Tuxedo Junction,1975-04-02,1975,185600,0,43,"a cappella,adult standards,vocal jazz",...,1,-9.364,0,0.0412,0.68200,0.000005,0.1360,0.4700,111.676,"[vocal jazz, adult standards, acappella]"
66999,2dogRElUKV20C2khRHqjRc,The Manhattan Transfer,2eIsJ2wadb9y03M0lQsvft,Java Jive,1975-04-02,1975,168400,0,29,"a cappella,adult standards,vocal jazz",...,6,-19.200,1,0.0407,0.96400,0.000000,0.0975,0.3570,102.632,"[vocal jazz, adult standards, acappella]"


In [22]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85263 entries, 0 to 85262
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         85263 non-null  object 
 1   artist_name       85263 non-null  object 
 2   id                85263 non-null  object 
 3   name              85263 non-null  object 
 4   release_date      85263 non-null  object 
 5   year              85263 non-null  int64  
 6   duration_ms       85263 non-null  int64  
 7   explicit          85263 non-null  int64  
 8   popularity        85263 non-null  int64  
 9   artist_genre      74030 non-null  object 
 10  danceability      85263 non-null  float64
 11  energy            85263 non-null  float64
 12  key               85263 non-null  int64  
 13  loudness          85263 non-null  float64
 14  mode              85263 non-null  int64  
 15  speechiness       85263 non-null  float64
 16  acousticness      85263 non-null  float6

In [23]:
new_df.head()

,artist_id,artist_name,id,name,release_date,year,duration_ms,explicit,popularity,artist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,context
0,05Q9xndTxhXhD5trpmTtfU,Carlos Gardel,71bkiosdVssjMrVDbdeG7n,Volver,1931-01-01,1931,172613,0,48,"latin classical,nueva cancion,tango",...,1,-9.108,0,0.0482,0.981,0.000017,0.3000,0.536,69.804,"[tango, nueva cancion, latin classical]"
1,05Q9xndTxhXhD5trpmTtfU,Carlos Gardel,3auaAuDjfudSv259S2UVlo,Mano A Mano,1931-01-01,1931,187173,0,35,"latin classical,nueva cancion,tango",...,4,-11.163,0,0.1340,0.988,0.000968,0.1770,0.613,121.847,"[tango, nueva cancion, latin classical]"
2,05Q9xndTxhXhD5trpmTtfU,Carlos Gardel,0z12YPLzCh2yZgwT6OGhjn,Tomo Y Obligo,1931-01-01,1931,132453,0,27,"latin classical,nueva cancion,tango",...,1,-9.570,0,0.1660,0.985,0.000003,0.0902,0.818,118.492,"[tango, nueva cancion, latin classical]"
3,0F4mauf7d7HcNLyBboIlq3,Llorenç Villalonga i Pons,0QHzk8M0hPmXupwDS3FDjR,Chapter 21.2 & Chapter 22.1 - Mort de dama,1931,1931,205458,0,0,None,...,7,-18.672,0,0.9490,0.913,0.000000,0.1580,0.727,122.137,[nan]
4,0F4mauf7d7HcNLyBboIlq3,Llorenç Villalonga i Pons,0SKuJceT2mC1a4uMnt2iy1,Chapter 11.2 - Mort de dama,1931,1931,393400,0,0,None,...,10,-18.886,0,0.9640,0.883,0.000000,0.1310,0.633,80.212,[nan]


In [24]:
new_df.to_csv('data/cleaned_data.csv', index=False)

In [25]:
def clean_text(text):
#     text = text.replace(r"[", "").replace("]", "").replace("'", "")
    lst = str(text).split(', ')
    lst1 = lst[:5]
    lst1.extend([np.nan] * (5 - len(lst1)))
    return lst1[0], lst1[1], lst1[2], lst1[3], lst1[4]


raw_df['artist_1'], raw_df['artist_2'], raw_df['artist_3'], raw_df['artist_4'], raw_df['artist_5'] = zip(*raw_df['artists_clean'].map(clean_text))
raw_df.head()

KeyError: 'artists_clean'